In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import numpy as np
np.bool = np.bool_

In [3]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [4]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [5]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [6]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

25/03/13 16:56:28 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/03/13 16:56:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cf1fc7fa-4335-46b2-b910-ed89b986b73a;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 193ms :: artifacts dl 6ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.spark#spark-avro_2.1

In [7]:
base_path = f"{hdfs_lakehouse_base_path}/bronze/trump_btc"

# List of folder names (from the structure provided)
folder_names = ["BTC", "Trump"]

# Dictionary to store DataFrames
dataframes = {}

# Loop through each folder and read the CSV file inside it
for folder_name in folder_names:
    # Construct the file path
    file_path = os.path.join(base_path, folder_name)
    
    # Read the CSV file into a DataFrame
    dataframes[folder_name] = (spark.read
                                        .option("header", "true")
                                        .option("inferSchema", "true")
                                        .csv(file_path))
    print(f"Loaded DataFrame for {folder_name}_raw")

# Example: Accessing one of the DataFrames
btc_df = dataframes["BTC"]
trump_df=dataframes["Trump"]


Loaded DataFrame for BTC_raw
Loaded DataFrame for Trump_raw


In [12]:
btc_df.show()

+------------+----+----+----+-----+------+
|   Timestamp|Open|High| Low|Close|Volume|
+------------+----+----+----+-----+------+
|1.32541206E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541212E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541218E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541224E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254123E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541236E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541242E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541248E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541254E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254126E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541266E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541272E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541278E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541284E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254129E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541296E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541302E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541308E9|4.58|4.58|4.58| 4.58|   0.0|
|1.32541314E9|4.58|4.58|4.58| 4.58|   0.0|
| 1.3254132E9|4.58|4.58|4.58| 4.58|   0.0|
+----------

In [9]:
trump_df.toPandas().head()

,date,favorites,id,isRetweet,retweets,text
0,1730689722303,0,113422481640840196,True,0,"""<p>RT <span class=""""h-card""""><a href=""""https://truthsocial.com/@TeamTrump"""" class=""""u-url mention"""">@<span>TeamTrump</span></a></span>PRESIDENT TRUMP: I’d like to begin by asking a question. Are you better off now than you were 4 years ago?</p><p>With your vote on Tuesday"
1,1730676754118,12350,113421631757789522,False,3124,"""<p>KAMALA BROKE IT, BUT I WILL FIX IT! With your VOTE, inflation will END. The border will be SAFE &amp; SECURE. We will have PEACE across the globe. WITH YOUR VOTE, AMERICA WILL ENTER A NEW GOLDEN AGE! Vote TRUMP to MAKE AMERICA GREAT AGAIN! <a href=""""https://links.truthsocial.com/link/112559845980056031"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://</span><span class="""""""">swampthevoteusa.com/</span><span class=""""invisible""""></span></a></p>"""
2,1730657728247,0,113420384878459920,True,0,"""<p>RT <span class=""""h-card""""><a href=""""https://truthsocial.com/@TeamTrump"""" class=""""u-url mention"""">@<span>TeamTrump</span></a></span>PRESIDENT TRUMP: IN CONCLUSION"
3,1730611542332,17960,113417358038328332,False,5386,"<p>If Kamala wins, you are 3 days away from the start of a 1929-style economic depression. If I win, you are 3 days away from the best jobs, the biggest paychecks, and the brightest economic future the world has ever seen.</p><p>Kamala’s inflation nightmare has cost the typical American family $30,000 dollars in higher prices—and now, she wants to impose the largest tax hike in American history, and raise your taxes by $3,000 dollars a year. </p><p>I will massively cut taxes for workers and small businesses—and we will have NO TAX ON TIPS, NO TAX ON OVERTIME, and NO TAX ON SOCIAL SECURITY benefits!</p>"
4,1730509334162,8165,113410659723667193,False,1919,"<p>Patrick Morrisey, of West Virginia, has been a wonderful Attorney General, and will now be an incredible Governor! He will fight tirelessly to Grow the Economy, Stop Inflation, Secure our Border, Restore American Energy DOMINANCE, Strengthen our Military/Vets, and Protect our always under siege Second Amendment. Patrick will work closely with me to enact our America First Agenda, and is a FANTASTIC person to replace my friend and future U.S. Senator, Jim Justice. Patrick Morrisey is my friend, and has my Complete and Total Endorsement - HE WILL NOT LET YOU DOWN!</p>"


In [10]:
from pyspark.sql.functions import col, from_unixtime, date_format

# If the timestamps are in milliseconds, divide by 1000
btc_df_std = btc_df.withColumnRenamed("Timestamp", "date") \
                   .withColumn("date", date_format(from_unixtime(col("date")), "yyyy-MM-dd HH:mm:ss")) \
                   .dropna()  # Remove missing values



In [11]:
btc_df_std.show(5)

+-------------------+----+----+----+-----+------+
|               date|Open|High| Low|Close|Volume|
+-------------------+----+----+----+-----+------+
|2012-01-01 11:01:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:02:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:03:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:04:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:05:00|4.58|4.58|4.58| 4.58|   0.0|
+-------------------+----+----+----+-----+------+
only showing top 5 rows



In [ ]:
btc_df_std.tail(5)

In [ ]:
#spark.sql("DROP SCHEMA trump_btc CASCADE")

In [ ]:
spark.sql("CREATE SCHEMA IF NOT EXISTS trump_btc")

In [ ]:
#saving btc data it in silver
(btc_df_std.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/")
            .saveAsTable("trump_btc.BTC")
)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, date_format,regexp_replace
from pyspark.sql.types import StringType, BooleanType, DoubleType, LongType, TimestampType
import re

trump_df = trump_df.withColumn("date", col("date").cast("bigInt"))
trump_df = trump_df.withColumn("date", date_format(from_unixtime(col("date") / 1000), "yyyy-MM-dd HH:mm:ss"))
trump_df = trump_df.withColumn("text", regexp_replace(col("text"), "<[^>]+>", ""))


In [ ]:
trump_df_std=trump_df
trump_df_std.show()

In [ ]:
#saving btc data it in silver
(trump_df_std.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/trump_btc/trump/")
            .saveAsTable("trump_btc.trump")
)


In [ ]:
%%sparksql
SELECT *
FROM trump_btc.trump
LIMIT 10